In [1]:
import numpy as np
import onnx
import os
import glob
from onnx import numpy_helper
import tvm
from tvm import te
import tvm.relay as relay

In [2]:
# module path
mobilenet_path = "../mobilenet/mobilenetv2-7.onnx"

In [4]:
# load module param and mod
mobilenet_model = onnx.load(mobilenet_path)
input_name = "input"
input_shape = (1, 3, 224, 224)
input_type = "float32"

shape_dict = {input_name: input_shape}
print(shape_dict)
dtype_dict = {input_name: input_type}
print(dtype_dict)

{'input': (1, 3, 224, 224)}
{'input': 'float32'}


In [5]:
mod, params = relay.frontend.from_onnx(mobilenet_model, shape=shape_dict, dtype=dtype_dict)

In [6]:
# target = tvm.target.arm_cpu("rasp3b")
target = tvm.target.Target('llvm')
#  target = tvm.target.Target("llvm -device=arm_cpu -mtriple=aarch64-linux-gnu -mattr=+neon")
# target = tvm.target.intel_graphics()
# target = tvm.target.Target("opencl -device=intel_graphics")
# target = tvm.target.Target("opencl -device=mali")
print(target)

llvm -keys=cpu -link-params=0


In [7]:
target_host = tvm.target.Target('llvm')
# target_host = "llvm -mtriple=aarch64-linux-gnu"
print(target_host)

llvm -keys=cpu -link-params=0


In [8]:
with tvm.transform.PassContext(opt_level=3):
# with relay.build_config(opt_level=3):    
    module_lib = relay.build(mod, target=target, target_host=target_host, params=params, mod_name='mobilenet')
print(module_lib)

One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


In [9]:
# output lib path
lib_path = "../module/x64/mobilenet.so"

In [10]:
# export params path
param_path = "../module/x64/mobilenet.params"

In [11]:
module_lib.export_library(lib_path)
# cc="/usr/bin/aarch64-linux-gnu-g++"
# cc="/usr/bin/arm-linux-gnueabihf-g++
# tvm.contrib.cc.cross_compiler("aarch64-linux-gnu-g++")

In [12]:
with open(param_path, 'wb') as fo:
    fo.write(relay.save_param_dict(params))